In [1]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit, when}
import org.apache.spark.sql.types.{IntegerType, DoubleType}
import org.apache.spark.sql.DataFrame

println("all good")

all good


data is in the format:

In [2]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        close: Double,
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt,    s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1),          s(8).split(": ")(1))
}

val filename = "../Datasets/bitcoin_trading_data.txt"
val tradingInformation = sc.textFile(filename)
            .flatMap(l => l.split("]}},"))       // Splits into indiviual http calls
            .map(x => x.split(""": \[\{""")(1))  // Splits up between header info and data payload
            .flatMap(x => x.split("},"))         // Now we have row wise data. It is bit messy so need to clean it
            .map(x => x.replace("{", ""))
            .map(x => x.replace("\"", ""))
            .map(x => x.replace("]}}]", ""))     // Last response is special case
            .map(createTradingInfo)
            .cache()

tradingInformation.take(1).foreach(println)
println("All good")

TradingInfo(1595581200,9536.29,9514.4,9514.87,1017.91,9698013.33,9536.26,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_trading_data.txt
tradingInformation = MapPartitionsRDD[101] at map at <console>:48


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[101] at map at <console>:48

In [3]:
// Data Action

val df = spark.createDataFrame(tradingInformation).sort("time")
val df1 = df.withColumn("midPrice",($"high" + $"low")/2)

val ratio: Double = 0.9
val training_size = df1.count*ratio

val training_position = df1.take(training_size.toInt).last.getInt(0)

val training_set = df1.filter($"time" <= training_position).cache()
val validation_set = df1.filter($"time" > training_position).cache()

println("All good")

All good


df = [time: int, high: double ... 7 more fields]
df1 = [time: int, high: double ... 8 more fields]
ratio = 0.9
training_size = 12602.7
training_position = 1597734000
training_set = [time: int, high: double ... 8 more fields]
validation_set = [time: int, high: double ... 8 more fields]


[time: int, high: double ... 8 more fields]

In [4]:
def take_statistics_df(df: DataFrame): DataFrame = {
    val statistics = df.agg(mean("midPrice"), stddev("midPrice"), min("midPrice"), max("midPrice"),
                            mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
                            mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")).head()
    val midPrice_mean = statistics.getDouble(0)
    val midPrice_std = statistics.getDouble(1)
    val midPrice_min = statistics.getDouble(2)
    val midPrice_max = statistics.getDouble(3)
    val midPrice_range = midPrice_max - midPrice_min

    // // VolumeFrom
    val volumefrom_mean = statistics.getDouble(4)
    val volumefrom_std = statistics.getDouble(5)
    val volumefrom_min = statistics.getDouble(6)
    val volumefrom_max = statistics.getDouble(7)
    val volumefrom_range = volumefrom_max - volumefrom_min

    // // VolumeTo
    val volumeto_mean = statistics.getDouble(8)
    val volumeto_std = statistics.getDouble(9)
    val volumeto_min = statistics.getDouble(10)
    val volumeto_max = statistics.getDouble(11)
    val volumeto_range = volumeto_max - volumeto_min

    val new_dataset = df.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
                   .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
                   .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
                   .as("df_2")

    
    return new_dataset
}

println("All good")

take_statistics_df: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


All good


In [5]:
val training_df = take_statistics_df(training_set).cache()
val validation_df = take_statistics_df(validation_set).cache()

println("all good")

all good


training_df = [time: int, high: double ... 11 more fields]
validation_df = [time: int, high: double ... 11 more fields]


[time: int, high: double ... 11 more fields]

In [14]:
// Saving data as Text
// don't forget to delete the previous folder 
val data_path_training = "cleaned_data/training_set"
val save_training_df = training_df.withColumn("empty", typedLit(Seq(1)))
save_training_df.rdd.repartition(1).saveAsTextFile(data_path_training)

val data_path_validation = "cleaned_data/validation_set"
val save_validation_df = validation_df.withColumn("empty", typedLit(Seq(1)))
save_validation_df.rdd.repartition(1).saveAsTextFile(data_path_validation)

data_path_training = cleaned_data/training_set
save_training_df = [time: int, high: double ... 12 more fields]
data_path_validation = cleaned_data/validation_set
save_validation_df = [time: int, high: double ... 12 more fields]


[time: int, high: double ... 12 more fields]

In [7]:
def take_statistics_social(df: DataFrame): DataFrame = {
        val statistics = df.agg(mean("total_page_views"), stddev("total_page_views"), min("total_page_views"), max("total_page_views"),
                                     mean("fb_talking_about"), stddev("fb_talking_about"), min("fb_talking_about"), max("fb_talking_about"),
                                     mean("reddit_posts_per_hour"), stddev("reddit_posts_per_hour"), min("reddit_posts_per_hour"), max("reddit_posts_per_hour"), 
                                     mean("reddit_comments_per_hour"), stddev("reddit_comments_per_hour"), min("reddit_comments_per_hour"), max("reddit_comments_per_hour")).head()

        val pageViews_mean = statistics.getDouble(0)
        val pageViews_std = statistics.getDouble(1)
        val pageViews_min = statistics.getInt(2)
        val pageViews_max = statistics.getInt(3)
        val pageViews_range = pageViews_max - pageViews_min

        // // fbTalking
        val fbTalking_mean = statistics.getDouble(4)
        val fbTalking_std = statistics.getDouble(5)
        val fbTalking_min = statistics.getInt(6)
        val fbTalking_max = statistics.getInt(7)
        val fbTalking_range = fbTalking_max - fbTalking_min

        // // redditPosts
        val redditPosts_mean = statistics.getDouble(8)
        val redditPosts_std = statistics.getDouble(9)
        val redditPosts_min = statistics.getDouble(10)
        val redditPosts_max = statistics.getDouble(11)
        val redditPosts_range = redditPosts_max - redditPosts_min

        val redditComments_mean = statistics.getDouble(12)
        val redditComments_std = statistics.getDouble(13)
        val redditComments_min = statistics.getDouble(14)
        val redditComments_max = statistics.getDouble(15)
        val redditComments_range = redditComments_max - redditComments_min


        val new_df = df.withColumn("pageViewsNorm",(($"total_page_views" - pageViews_min)/pageViews_range).cast(DoubleType))
                       .withColumn("fbTalkingNorm",(($"fb_talking_about" - fbTalking_min)/fbTalking_range).cast(DoubleType))
                       .withColumn("redditPostsNorm",(($"reddit_posts_per_hour" - redditPosts_min)/redditPosts_range).cast(DoubleType))
                       .withColumn("redditCommentsNorm",(($"reddit_comments_per_hour" - redditComments_min)/redditComments_range).cast(DoubleType))
    
    return new_df
}

println("all good")

take_statistics_social: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


all good


In [12]:
// Social data is a colelction of various values from different social platforms 
// Explanation of the values can be found at: https://min-api.cryptocompare.com/documentation?key=Social&cat=historicalHourSocialStats


case class SocialData (time: Int,
                comments: Int,
                posts: Int,
                followers: Int,
                points: Int,
                overview_page_views: Int,
                analysis_page_views: Int,
                markets_page_views: Int,
                charts_page_views: Int,
                trades_page_views: Int, 
                forum_page_views: Int, 
                influence_page_views: Int, 
                total_page_views: Int, //
                fb_likes: Int,
                fb_talking_about: Int, //
                twitter_followers: Int,
                twitter_following: Int,
                twitter_lists: Int,
                twitter_favourites: Int,
                twitter_statuses: Int,
                reddit_subscribers: Int,
                reddit_active_users: Int,
                reddit_posts_per_hour: Double, //
                reddit_posts_per_day: Double,
                reddit_comments_per_hour: Double, //
                reddit_comments_per_day: Double,
                code_repo_stars: Int,
                code_repo_forks: Int,
                code_repo_subscribers: Int,
                code_repo_open_pull_issues: Int,
                code_repo_closed_pull_issues: Int,
                code_repo_open_issues: Int,
                code_repo_closed_issues: Int,
                code_repo_contributors: Int                     
               )

def createSocialData(data: String): SocialData = {
val s = data.split(", ")
return SocialData(extractAndConvertToInt(s(0)),    extractAndConvertToInt(s(1)),     extractAndConvertToInt(s(2)),    extractAndConvertToInt(s(3)),
              extractAndConvertToInt(s(4)),    extractAndConvertToInt(s(5)),     extractAndConvertToInt(s(6)),    extractAndConvertToInt(s(7)),
              extractAndConvertToInt(s(8)),    extractAndConvertToInt(s(9)),     extractAndConvertToInt(s(10)),   extractAndConvertToInt(s(11)),
              extractAndConvertToInt(s(12)),   extractAndConvertToInt(s(13)),    extractAndConvertToInt(s(14)),   extractAndConvertToInt(s(15)),
              extractAndConvertToInt(s(16)),   extractAndConvertToInt(s(17)),    extractAndConvertToInt(s(18)),   extractAndConvertToInt(s(19)),
              extractAndConvertToInt(s(20)),   extractAndConvertToInt(s(21)),    extractAndConvertToDouble(s(22)),extractAndConvertToDouble(s(23)),
              extractAndConvertToDouble(s(24)),extractAndConvertToDouble(s(25)), extractAndConvertToInt(s(26)),   extractAndConvertToInt(s(27)),
              extractAndConvertToInt(s(28)),   extractAndConvertToInt(s(29)),    extractAndConvertToInt(s(30)),   extractAndConvertToInt(s(31)),
              extractAndConvertToInt(s(32)),   extractAndConvertToInt(s(33)))
}

def extractAndConvertToInt(attribute: String): Int = {
return attribute.split(": ")(1).toInt
}

def extractAndConvertToDouble(attribute: String): Double = {
return attribute.split(": ")(1).toDouble
}


val filename_social = "../Datasets/bitcoin_social_data.txt"
val socialDataInformation = sc.textFile(filename_social)
    .flatMap(l => l.split("]},"))         // Splits into indiviual http calls
    .map(x => x.split(""": \[\{""")(1))   // Splits up between header info and data payload
    .flatMap(x => x.split("},"))          // Now we have row wise data. It is bit messy so need to clean it
    .map(x => x.replace("{", ""))
    .map(x => x.replace("\"", ""))
    .map(x => x.replace("}", ""))
    .map(x => x.replace("]]", ""))     // Last response is special case
    .map(createSocialData)
    .cache()    

    //         socialDataInformation.take(1).foreach(println)

// Merge TradingInfo with SocialData

var socialDataDf = spark.createDataFrame(socialDataInformation).sort("time")
val socialDatadf_shorten = socialDataDf.select("time", "total_page_views", "fb_talking_about", "reddit_posts_per_hour", "reddit_comments_per_hour")
val norm_social_df = take_statistics_social(socialDatadf_shorten)

val mergedDataDf_training = training_df.join(norm_social_df, Seq("time")).cache()
val mergedDataDf_validation = validation_df.join(norm_social_df, Seq("time")).cache()
mergedDataDf_training.printSchema()

println("All good")

root
 |-- time: integer (nullable = false)
 |-- high: double (nullable = false)
 |-- low: double (nullable = false)
 |-- open: double (nullable = false)
 |-- volumefrom: double (nullable = false)
 |-- volumeto: double (nullable = false)
 |-- close: double (nullable = false)
 |-- conversionType: string (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- midPrice: double (nullable = true)
 |-- midPriceNorm: double (nullable = true)
 |-- volumefromNorm: double (nullable = true)
 |-- volumetoNorm: double (nullable = true)
 |-- total_page_views: integer (nullable = false)
 |-- fb_talking_about: integer (nullable = false)
 |-- reddit_posts_per_hour: double (nullable = false)
 |-- reddit_comments_per_hour: double (nullable = false)
 |-- pageViewsNorm: double (nullable = true)
 |-- fbTalkingNorm: double (nullable = true)
 |-- redditPostsNorm: double (nullable = true)
 |-- redditCommentsNorm: double (nullable = true)

All good


defined class SocialData
filename_social = ../Datasets/bitcoin_social_data.txt
socialDataInformation = MapPartitionsRDD[282] at map at <console>:105
socialDataDf = [time: int, comments: int ... 32 more fields]
socialDatadf_shorten = [time: int, total_page_views: int ... 3 more fields]
norm_social_df = [time: int, total_page_views: int ... 7 more fields]
mergedDataDf_training = [time: int, high: double ... 19 more fields]


createSocialData: (data: String)SocialData
extractAndConvertToInt: (attribute: String)Int
extractAndConvertToDouble: (attribute: String)Double
mergedD...


[time: int, high: double ... 19 more fields]

In [13]:
// Saving data as Text
// don't forget to delete the previous folder 
val data_path_training_social = "cleaned_data_socialMedia/training_set"
val save_training_df_social = mergedDataDf_training.withColumn("empty", typedLit(Seq(1)))
save_training_df_social.rdd.repartition(1).saveAsTextFile(data_path_training_social)

val data_path_validation_social = "cleaned_data_socialMedia/validation_set"
val save_validation_df_social = mergedDataDf_validation.withColumn("empty", typedLit(Seq(1)))
save_validation_df_social.rdd.repartition(1).saveAsTextFile(data_path_validation_social)

data_path_training_social = cleaned_data_socialMedia/training_set
save_training_df_social = [time: int, high: double ... 20 more fields]
data_path_validation_social = cleaned_data_socialMedia/validation_set
save_validation_df_social = [time: int, high: double ... 20 more fields]


[time: int, high: double ... 20 more fields]